In [2]:
## This file uses all generated data, so it needs to be run after the files that use kaggle data

In [72]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [79]:
coach_features = pd.read_csv('data/2023/generated_data/coach_features.csv')
stationary_probs_all = pd.read_csv('data/2023/generated_data/stationary_probabilities.csv')
stationary_probs = stationary_probs_all[stationary_probs_all['Season'] >= 2003]
team_season_features = pd.read_csv('data/2023/generated_data/team_season_features.csv')
additional_team_features = pd.read_csv('data/2023/generated_data/additional_team_features.csv')
preseason_rankings = pd.read_csv('data/2023/generated_data/preseason_rankings.csv')

In [80]:
additional_team_features.drop_duplicates(inplace=True)

In [81]:
preseason_rankings.columns

Index(['TeamID', 'Season', 'school', 'preseason_pts'], dtype='object')

In [82]:
idx = ['Season', 'TeamID']
features_all = team_season_features.merge(stationary_probs, on=idx)\
.merge(coach_features, on=idx)\
.merge(additional_team_features, on=idx)\
.merge(preseason_rankings, on=idx, how='left')
features_all['preseason_pts'] = features_all['preseason_pts'].fillna(0)
features_all.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement,Unnamed: 0,pi_i,school_x,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,rk,school_y,returning_min_pct,returning_score_pct,school,preseason_pts
0,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,6.357143,0.320144,14.857143,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,Y10,5.0,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0,5409,0.004730,alabama,0,1,0,0.660,4,0,0,0,1,alabama,80.5,78.0,alabama,1295.0
1,2003,1112,28,25,3,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,7.035714,0.350534,17.535714,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,Z01,10.0,5.0,2.0,1.0,12.0,2.0,8.0,6.0,3.0,3.0,0.0,5416,0.007921,arizona,1,0,0,0.738,24,12,5,1,1,arizona,98.4,99.0,arizona,1773.0
2,2003,1113,29,18,11,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,4.000000,0.317808,17.551724,26.206897,0.669737,14.000000,65.671136,115.334912,103.952243,0.786348,-0.165659,Z10,4.0,2.0,10.0,1.0,14.0,32.0,30.0,-2.0,32.0,24.0,-8.0,5417,0.004969,arizona-state,0,0,0,0.520,3,0,0,0,1,arizona-state,73.2,66.6,arizona state,4.0
3,2003,1120,30,19,11,0.633333,70.100000,65.566667,4.533333,18.358891,6.0,4.0,2.0,2.0,16.333333,5.633333,0.344898,13.066667,19.733333,0.662162,15.900000,64.715520,108.023523,101.505949,0.717067,-0.083733,W10,4.0,1.0,8.0,3.0,12.0,36.0,31.0,-5.0,45.0,45.0,0.0,5423,0.004403,auburn,0,0,0,0.617,8,3,0,0,1,auburn,67.8,70.9,NaN,0.0
4,2003,1122,27,20,7,0.740741,69.703704,67.703704,2.000000,12.584529,9.0,0.0,8.0,1.0,17.185185,6.000000,0.349138,16.592593,24.000000,0.691358,15.222222,63.424000,108.269069,104.582414,0.599878,0.140863,W13,1.0,1.0,1.0,6.0,2.0,73.0,95.0,22.0,123.0,143.0,20.0,5424,0.003027,austin-peay,0,0,0,0.522,2,0,0,0,1,austin-peay,73.5,62.7,NaN,0.0


In [83]:
print(features_all.shape)

(1336, 55)


In [84]:
features_all[features_all['Season']==2023]

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement,Unnamed: 0,pi_i,school_x,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,rk,school_y,returning_min_pct,returning_score_pct,school,preseason_pts
1268,2023,1104,34,29,5,0.852941,82.176471,68.500000,13.676471,17.443557,14.0,0.0,6.0,0.0,29.764706,10.058824,0.337945,16.588235,22.852941,0.725869,13.588235,70.601035,115.107979,94.059010,0.926362,-0.073421,X01,14.0,9.0,4.0,1.0,18.0,2.0,3.0,1.0,3.0,5.0,2.0,12302,0.007706,alabama,0,0,1,0.689,6,1,0,0,1,alabama,40.7,41.0,alabama,281.0
1269,2023,1112,34,28,6,0.823529,82.676471,71.500000,11.176471,14.178423,13.0,2.0,3.0,1.0,22.500000,8.588235,0.381699,15.323529,21.617647,0.708844,13.117647,70.331294,117.751863,101.611150,0.882907,-0.059378,X02,12.0,7.0,2.0,4.0,14.0,11.0,9.0,-2.0,11.0,10.0,-1.0,12309,0.006254,arizona,0,0,1,0.859,2,1,0,0,1,arizona,45.3,44.6,arizona,543.0
1270,2023,1113,34,22,12,0.647059,70.323529,67.705882,2.617647,13.847945,12.0,5.0,5.0,3.0,23.911765,7.500000,0.313653,12.823529,18.617647,0.688784,11.529412,67.197176,104.084801,99.975904,0.628936,0.018123,Z11a,4.0,4.0,7.0,5.0,11.0,65.0,67.0,2.0,61.0,66.0,5.0,12310,0.004105,arizona-state,0,0,0,0.580,4,0,0,0,1,arizona-state,48.8,40.4,NaN,0.0
1271,2023,1116,33,20,13,0.606061,74.424242,67.393939,7.030303,14.819080,6.0,3.0,3.0,5.0,16.060606,5.090909,0.316981,15.212121,22.060606,0.689560,12.515152,67.596606,110.095243,99.201117,0.799030,-0.192969,Z08,6.0,3.0,11.0,2.0,17.0,18.0,29.0,11.0,19.0,27.0,8.0,12313,0.004993,arkansas,0,1,0,0.730,6,3,0,0,1,arkansas,16.6,13.0,arkansas,1026.0
1272,2023,1120,32,20,12,0.625000,72.718750,67.062500,5.656250,13.823914,6.0,2.0,2.0,6.0,20.718750,6.500000,0.313725,14.593750,20.781250,0.702256,11.875000,66.136600,108.604032,102.193442,0.755165,-0.130165,Y09,7.0,1.0,10.0,2.0,17.0,32.0,30.0,-2.0,26.0,24.0,-2.0,12316,0.004629,auburn,0,1,0,0.666,12,5,1,0,1,auburn,63.2,56.6,auburn,623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,2023,1433,34,27,7,0.794118,71.441176,62.911765,8.529412,10.968498,12.0,3.0,4.0,1.0,17.529412,6.088235,0.347315,15.588235,22.352941,0.697368,12.852941,65.774118,109.225707,95.481197,0.854275,-0.060158,Z12,3.0,1.0,2.0,5.0,5.0,67.0,78.0,11.0,76.0,75.0,-1.0,12619,0.004097,virginia-commonwealth,0,0,0,0.611,3,0,0,0,1,virginia-commonwealth,41.4,36.1,NaN,0.0
1332,2023,1436,32,22,10,0.687500,72.156250,66.718750,5.437500,15.668315,10.0,1.0,2.0,3.0,24.031250,8.687500,0.361508,11.093750,15.781250,0.702970,8.875000,62.652893,113.936345,105.239509,0.748348,-0.060848,W15,1.0,1.0,3.0,7.0,4.0,114.0,135.0,21.0,118.0,130.0,12.0,12622,0.003231,vermont,0,0,0,0.714,5,0,0,0,1,vermont,40.2,33.5,NaN,0.0
1333,2023,1438,32,25,7,0.781250,67.812500,60.250000,7.562500,11.717875,10.0,1.0,4.0,2.0,19.281250,6.812500,0.353323,12.937500,18.468750,0.700508,8.125000,60.055133,112.618435,100.065056,0.838183,-0.056933,X04,4.0,2.0,4.0,3.0,8.0,30.0,15.0,-15.0,36.0,16.0,-20.0,12624,0.004374,virginia,0,0,1,0.723,11,4,1,1,1,virginia,87.8,91.4,virginia,462.0
1334,2023,1452,33,19,14,0.575758,76.303030,71.030303,5.272727,15.531537,6.0,4.0,2.0,3.0,20.878788,7.272727,0.348331,16.909091,22.818182,0.741036,12.454545,66.963006,113.727339,106.149725,0.730279,-0.154522,X09,6.0,2.0,11.0,3.0,17.0,24.0,20.0,-4.0,17.0,14.0,-3.0,12636,0.004951,west-virginia,0,0,0,0.690,26,9,2,0,1,west-virginia,17.1,11.5,NaN,0.0


In [85]:
features_all[features_all['Season'] == 2023].sort_values(by='pi_i', ascending=False)[['Season','school_y','pi_i']].head(50)

,Season,school_y,pi_i
1268,2023,alabama,0.007706
1286,2023,houston,0.007344
1326,2023,ucla,0.007121
1277,2023,connecticut,0.006619
1323,2023,texas,0.006538
1315,2023,purdue,0.006366
1322,2023,tennessee,0.006319
1269,2023,arizona,0.006254
1293,2023,kansas,0.006052
1284,2023,gonzaga,0.005934


In [86]:
# removed avg height and avg year this year
features = features_all[['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for', 
              'avg_points_against', 'avg_game_margin', 'std_game_margin', 
              'non_home_wins', 'home_losses', 'close_wins', 'close_losses', 
              '3FGA_pg', '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 
              'off_eff', 'def_eff', 'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
              'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
              'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
              'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
              'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
              'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
              'returning_min_pct', 'returning_score_pct', 'preseason_pts']]

In [87]:
print(features.shape)
features.head()

(1336, 44)


,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3_pct,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,net_improvement,final_pom,pom_improvement,pi_i,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
0,2003,1104,Y10,28,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,0.320144,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,5.0,1.0,6.0,5.0,11.0,38.0,-11.0,33.0,6.0,0.004730,0,1,0,0.660,4,0,0,0,80.5,78.0,1295.0
1,2003,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.007921,1,0,0,0.738,24,12,5,1,98.4,99.0,1773.0
2,2003,1113,Z10,29,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,0.317808,26.206897,0.669737,14.000000,65.671136,115.334912,103.952243,0.786348,-0.165659,4.0,2.0,10.0,1.0,14.0,32.0,-2.0,32.0,-8.0,0.004969,0,0,0,0.520,3,0,0,0,73.2,66.6,4.0
3,2003,1120,W10,30,0.633333,70.100000,65.566667,4.533333,18.358891,6.0,4.0,2.0,2.0,16.333333,0.344898,19.733333,0.662162,15.900000,64.715520,108.023523,101.505949,0.717067,-0.083733,4.0,1.0,8.0,3.0,12.0,36.0,-5.0,45.0,0.0,0.004403,0,0,0,0.617,8,3,0,0,67.8,70.9,0.0
4,2003,1122,W13,27,0.740741,69.703704,67.703704,2.000000,12.584529,9.0,0.0,8.0,1.0,17.185185,0.349138,24.000000,0.691358,15.222222,63.424000,108.269069,104.582414,0.599878,0.140863,1.0,1.0,1.0,6.0,2.0,73.0,22.0,123.0,20.0,0.003027,0,0,0,0.522,2,0,0,0,73.5,62.7,0.0


## Tourney games

In [107]:
tourney_games_all = pd.read_csv('data/2023/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_games = tourney_games_all[tourney_games_all['Season'] >= 2003]
tourney_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [108]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'preseason_pts']

In [110]:
def process_df(tourney_games, flip):
  tourney_games['teams'] = [(t1, t2) if not flip else (t2, t1) for t1, t2 in \
                            zip(tourney_games['WTeamID'], tourney_games['LTeamID'])]
  tourney_games['t1_score_diff'] = [t1-t2 if not flip else t2-t1 for t1, t2 in \
                            zip(tourney_games['WScore'], tourney_games['LScore'])]
  tourney_games[['team1','team2']] = pd.DataFrame(tourney_games['teams'].tolist(), index=tourney_games.index)
  tourney_results = tourney_games[['Season', 'team1', 'team2', 't1_score_diff']]
  tourney_results['is_t1_win'] = [diff > 0 for diff in tourney_results['t1_score_diff']]
  matchup_features_all = tourney_results.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
  .merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])
  for col in difference_cols:
    matchup_features_all['diff_'+col] = matchup_features_all[col+'_1'] - matchup_features_all[col+'_2']
  diff_cols = [col for col in matchup_features_all.columns if 'diff' in col]
  matchup_cols = ['Season', 'TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols
  matchup_features_basic = matchup_features_all[matchup_cols]
  return matchup_features_basic

In [111]:
res_1 = process_df(tourney_games.copy(), False)
res_2 = process_df(tourney_games.copy(), True)

In [112]:
res_1.head()

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2003,1421,1411,X16b,X16a,8,-1,-0.151724,-1.593103,7.614943,-9.208046,5.205395,-4.0,-1.0,1.0,1.0,-0.500000,0.039433,-7.135632,0.142815,0.973563,-1.447619,-2.263666,12.631956,-0.387769,0.236045,0.0,0.0,5.0,-1.0,5.0,-31.0,11.0,5.0,8.0,-0.000391,-0.029,0,0,0,0,-11.2,-7.0,0.0
1,2003,1112,1436,Z01,Z16,29,-1,0.237685,17.421182,7.112069,10.309113,-0.052399,1.0,0.0,0.0,-2.0,4.588670,0.009777,5.448276,0.043580,0.716749,10.195649,8.539513,-4.232186,0.207207,0.030478,10.0,5.0,1.0,-8.0,11.0,-142.0,-35.0,-141.0,-38.0,0.004756,0.285,23,12,5,1,37.2,46.1,1773.0
2,2003,1112,1211,Z01,Z09,1,-3,0.150922,8.149770,2.056452,6.093318,-0.615673,-1.0,-1.0,-3.0,0.0,1.006912,-0.025101,0.354839,-0.019776,0.237327,7.062525,0.120885,-5.971985,0.090526,0.060395,5.0,2.0,-3.0,-2.0,2.0,-41.0,3.0,-38.0,12.0,0.002800,-0.046,20,10,5,1,NaN,NaN,1392.0
3,2003,1112,1323,Z01,Z05,17,-3,0.183180,5.117512,-0.943548,6.061060,-2.866029,3.0,-1.0,-2.0,0.0,-1.702765,-0.030207,2.161290,-0.058458,2.011521,4.466768,0.959411,-7.112776,0.098797,0.084383,5.0,3.0,-4.0,-2.0,1.0,-12.0,16.0,-19.0,10.0,0.001456,0.073,19,11,5,1,43.2,49.7,1715.0
4,2003,1113,1272,Z10,Z07,13,0,-0.172414,1.448276,3.344828,-1.896552,6.116003,-3.0,1.0,1.0,-1.0,-7.482759,-0.030989,3.310345,0.016122,0.206897,-1.004653,4.182093,5.603289,-0.062388,-0.110026,0.0,0.0,7.0,-2.0,7.0,6.0,-16.0,12.0,-37.0,-0.000375,-0.201,-3,-3,-1,0,NaN,NaN,2.0


In [113]:
res_2.head()

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2003,1411,1421,X16a,X16b,-8,1,0.151724,1.593103,-7.614943,9.208046,-5.205395,4.0,1.0,-1.0,-1.0,0.500000,-0.039433,7.135632,-0.142815,-0.973563,1.447619,2.263666,-12.631956,0.387769,-0.236045,0.0,0.0,-5.0,1.0,-5.0,31.0,-11.0,-5.0,-8.0,0.000391,0.029,0,0,0,0,11.2,7.0,0.0
1,2003,1436,1112,Z16,Z01,-29,1,-0.237685,-17.421182,-7.112069,-10.309113,0.052399,-1.0,0.0,0.0,2.0,-4.588670,-0.009777,-5.448276,-0.043580,-0.716749,-10.195649,-8.539513,4.232186,-0.207207,-0.030478,-10.0,-5.0,-1.0,8.0,-11.0,142.0,35.0,141.0,38.0,-0.004756,-0.285,-23,-12,-5,-1,-37.2,-46.1,-1773.0
2,2003,1211,1112,Z09,Z01,-1,3,-0.150922,-8.149770,-2.056452,-6.093318,0.615673,1.0,1.0,3.0,0.0,-1.006912,0.025101,-0.354839,0.019776,-0.237327,-7.062525,-0.120885,5.971985,-0.090526,-0.060395,-5.0,-2.0,3.0,2.0,-2.0,41.0,-3.0,38.0,-12.0,-0.002800,0.046,-20,-10,-5,-1,NaN,NaN,-1392.0
3,2003,1323,1112,Z05,Z01,-17,3,-0.183180,-5.117512,0.943548,-6.061060,2.866029,-3.0,1.0,2.0,0.0,1.702765,0.030207,-2.161290,0.058458,-2.011521,-4.466768,-0.959411,7.112776,-0.098797,-0.084383,-5.0,-3.0,4.0,2.0,-1.0,12.0,-16.0,19.0,-10.0,-0.001456,-0.073,-19,-11,-5,-1,-43.2,-49.7,-1715.0
4,2003,1272,1113,Z07,Z10,-13,0,0.172414,-1.448276,-3.344828,1.896552,-6.116003,3.0,-1.0,-1.0,1.0,7.482759,0.030989,-3.310345,-0.016122,-0.206897,1.004653,-4.182093,-5.603289,0.062388,0.110026,0.0,0.0,-7.0,2.0,-7.0,-6.0,16.0,-12.0,37.0,0.000375,0.201,3,3,1,0,NaN,NaN,-2.0


In [114]:
matchup_features_basic = pd.concat([res_1,res_2],ignore_index=True)

In [115]:
print(matchup_features_basic.shape)

(2496, 44)


In [116]:
matchup_features_basic.to_csv('data/2023/model_data/matchup_features_v2.csv', index=False)

In [136]:
matchup_features_basic.head()

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2003,1411,1421,X16a,X16b,-8,1,0.151724,1.593103,-7.614943,9.208046,-5.205395,4.0,1.0,-1.0,-1.0,0.500000,-0.039433,7.135632,-0.142815,-0.973563,1.447619,2.263666,-12.631956,0.387769,-0.236045,0.0,0.0,-5.0,1.0,-5.0,31.0,-11.0,-5.0,-8.0,0.000391,0.029,0,0,0,0,11.2,7.0,0.0
1,2003,1436,1112,Z16,Z01,-29,1,-0.237685,-17.421182,-7.112069,-10.309113,0.052399,-1.0,0.0,0.0,2.0,-4.588670,-0.009777,-5.448276,-0.043580,-0.716749,-10.195649,-8.539513,4.232186,-0.207207,-0.030478,-10.0,-5.0,-1.0,8.0,-11.0,142.0,35.0,141.0,38.0,-0.004756,-0.285,-23,-12,-5,-1,-37.2,-46.1,-1773.0
2,2003,1323,1112,Z05,Z01,-17,3,-0.183180,-5.117512,0.943548,-6.061060,2.866029,-3.0,1.0,2.0,0.0,1.702765,0.030207,-2.161290,0.058458,-2.011521,-4.466768,-0.959411,7.112776,-0.098797,-0.084383,-5.0,-3.0,4.0,2.0,-1.0,12.0,-16.0,19.0,-10.0,-0.001456,-0.073,-19,-11,-5,-1,-43.2,-49.7,-1715.0
3,2003,1113,1272,Z10,Z07,13,0,-0.172414,1.448276,3.344828,-1.896552,6.116003,-3.0,1.0,1.0,-1.0,-7.482759,-0.030989,3.310345,0.016122,0.206897,-1.004653,4.182093,5.603289,-0.062388,-0.110026,0.0,0.0,7.0,-2.0,7.0,6.0,-16.0,12.0,-37.0,-0.000375,-0.201,-3,-3,-1,0,NaN,NaN,2.0
4,2003,1113,1242,Z10,Z02,-32,-1,-0.145977,-5.367816,2.805747,-8.173563,-0.937985,-4.0,2.0,2.0,2.0,-1.547126,-0.021814,2.073563,0.003991,-0.900000,-4.666144,0.040565,9.297038,-0.157862,0.011885,-5.0,-1.0,4.0,0.0,-1.0,26.0,-7.0,28.0,-10.0,-0.002805,-0.285,-11,-9,-4,0,11.0,7.0,-1711.0


## 2023 Matchups

In [50]:
seeds = pd.read_csv("data/2023/kaggle_data/MNCAATourneySeeds.csv")


In [51]:
curr = seeds[seeds['Season']==2023]

In [52]:
from itertools import combinations


In [53]:
# get all two-pair combinations of column 'A' as a list
combos = list(combinations(seeds['TeamID'], 2))

unique_pairs = set(frozenset(pair) for pair in combos)


# create a new dataframe from the combinations list
combo_df = pd.DataFrame(unique_pairs, columns=['team1', 'team2'])

In [54]:
combo_df['Season']=2023

In [57]:
combo_df = combo_df[combo_df['team1'] != combo_df['team2']]

In [59]:
features[features['Season']==2023]

,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,...,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
1268,2023,1104,X01,34,0.852941,82.176471,68.500000,13.676471,17.443557,14.0,...,0,1,0.689,6,1,0,0,40.7,41.0,281.0
1269,2023,1112,X02,34,0.823529,82.676471,71.500000,11.176471,14.178423,13.0,...,0,1,0.859,2,1,0,0,45.3,44.6,543.0
1270,2023,1113,Z11a,34,0.647059,70.323529,67.705882,2.617647,13.847945,12.0,...,0,0,0.580,4,0,0,0,48.8,40.4,0.0
1271,2023,1116,Z08,33,0.606061,74.424242,67.393939,7.030303,14.819080,6.0,...,1,0,0.730,6,3,0,0,16.6,13.0,1026.0
1272,2023,1120,Y09,32,0.625000,72.718750,67.062500,5.656250,13.823914,6.0,...,1,0,0.666,12,5,1,0,63.2,56.6,623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,2023,1433,Z12,34,0.794118,71.441176,62.911765,8.529412,10.968498,12.0,...,0,0,0.611,3,0,0,0,41.4,36.1,0.0
1332,2023,1436,W15,32,0.687500,72.156250,66.718750,5.437500,15.668315,10.0,...,0,0,0.714,5,0,0,0,40.2,33.5,0.0
1333,2023,1438,X04,32,0.781250,67.812500,60.250000,7.562500,11.717875,10.0,...,0,1,0.723,11,4,1,1,87.8,91.4,462.0
1334,2023,1452,X09,33,0.575758,76.303030,71.030303,5.272727,15.531537,6.0,...,0,0,0.690,26,9,2,0,17.1,11.5,0.0


In [60]:
combo_df.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])

,team1,team2,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,...,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,preseason_pts
0,1222,1175.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
1,1222,1191.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
2,1222,1286.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
3,1222,1279.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
4,1222,1151.0,2023,1222,Y01,34,0.911765,75.029412,56.500000,18.529412,...,0,0,0.686,19,6,2,0,44.1,36.0,1404.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11919,1159,1135.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11920,1159,1247.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11921,1159,1199.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0
11922,1159,1183.0,2023,1159,Y15,33,0.757576,78.151515,69.242424,8.909091,...,0,0,0.539,4,0,0,0,69.2,65.4,0.0


In [61]:
# 2023 matchups

bracket_data = combo_df.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])

In [62]:
bracket_data.shape

(2278, 89)

In [63]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'preseason_pts']
for col in difference_cols:
    bracket_data['diff_'+col] = bracket_data[col+'_1'] - bracket_data[col+'_2']
diff_cols = [col for col in bracket_data.columns if 'diff' in col]
matchup_cols = ['Season', 'TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols
bracket_data = bracket_data[matchup_cols]

In [64]:
bracket_data.shape

(2278, 43)

In [65]:
bracket_data.to_csv('data/2023/bracket_data/bracket_data.csv', index=False)

In [66]:
bracket_data.head(20)

,Season,TeamID_1,TeamID_2,Seed_1,Seed_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,...,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_preseason_pts
0,2023,1222,1286,Y01,W14,2,0.193015,3.998162,-10.437500,14.435662,...,-4.0,0.003994,0.027,17,6,2,0,-8.1,-19.2,1404.0
1,2023,1280,1286,Y11a,W14,1,-0.082386,-5.122159,-5.937500,0.815341,...,-4.0,0.000765,0.083,2,0,0,0,-6.7,-13.3,0.0
2,2023,1394,1286,X16b,W14,-3,-0.063578,7.244612,5.993534,1.251078,...,15.0,-0.000962,0.022,0,0,0,0,32.3,30.0,0.0
3,2023,1277,1286,W07,W14,-1,-0.105847,-0.805444,0.449597,-1.255040,...,11.0,0.000938,0.052,23,14,8,1,1.9,-5.0,35.0
4,2023,1243,1286,W03,W14,0,0.000000,4.468750,2.093750,2.375000,...,4.0,0.001527,0.060,-1,0,0,0,-27.0,-29.7,0.0
5,2023,1224,1286,Z16,W14,-1,-0.105847,2.581653,5.675403,-3.093750,...,2.0,-0.001085,-0.233,-1,0,0,0,-5.4,-8.0,0.0
6,2023,1361,1286,X05,W14,0,0.093750,0.312500,-3.125000,3.437500,...,3.0,0.001674,0.101,2,0,0,0,15.0,13.7,394.0
7,2023,1344,1286,W11,W14,0,-0.062500,7.062500,4.375000,2.687500,...,-20.0,0.000969,-0.057,5,1,0,0,-26.7,-28.3,0.0
8,2023,1401,1286,Y07,W14,2,0.016544,2.204044,-0.731618,2.935662,...,10.0,0.001476,-0.038,7,4,0,0,13.4,5.6,112.0
9,2023,1369,1286,X16a,W14,1,-0.203598,6.180871,10.759470,-4.578598,...,-15.0,-0.001104,-0.196,-1,0,0,0,-16.6,-21.2,0.0
